<hr style="color:green" />
<h1 style="color:green">COSC2673 Assignment 2: Image Classification for Cancerous Cells</h1>
<h2 style="color:green">File 20: PyTorch First CNN model test on Main data, using AlexNet Config</h2>
<hr style="color:green" />

<p>
In this file, Train a basic Convolutional Neural Network (CNN) with Pytorch, using the AlexNet configuration. The AlexNet configuration contains 5 convolutional layers, 3 max pooling and an average pooling layer, and in the end it has 3 fully-connected layers, the last of which classifies images in 1000 different classes.
</p>
<p>
The CNN implementation from file 10 runs but has terrible precision. Try to fix here
</p>

In [1]:
import pandas as pd
import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image


c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Configure this script as to whether it runs on Google Colab, or locally

In [2]:
# When on Google Colab, running full training, change both to true. Locally, advised set both to false
isGoogleColab = False
useFullData = False

In [3]:
# In local, the base directory is the current directory
baseDirectory = "./"

if isGoogleColab:
    from google.colab import drive
    
    # If this is running on Google colab, assume the notebook runs in a "COSC2673" folder, which also contains the data files 
    # in a subfolder called "image_classification_data"
    drive.mount("/content/drive")
    !ls /content/drive/'My Drive'/COSC2673/

    # Import the directory so that custom python libraries can be imported
    import sys
    sys.path.append("/content/drive/MyDrive/COSC2673/")

    # Set the base directory to the Google Drive specific folder
    baseDirectory = "/content/drive/MyDrive/COSC2673/"

Import the custom python files that contain reusable code

In [4]:
import data_basic_utility as dbutil
import graphing_utility as graphutil
import statistics_utility as statsutil

import a2_utility as a2util
import pytorch_utility as ptutil
from pytorch_utility import CancerBinaryDataset
from pytorch_utility import CancerCellTypeDataset


# randomSeed = dbutil.get_random_seed()
randomSeed = 266305
print("Random Seed: " + str(randomSeed))

Random Seed: 266305


In [5]:
# this file should have previously been created in the root directory
dfImages = pd.read_csv(baseDirectory + "images_main.csv")

In [6]:
# Get The training Split and the Validation Split
dfImagesTrain = dfImages[dfImages["trainValTest"] == 0].reset_index()
dfImagesVal = dfImages[dfImages["trainValTest"] == 1].reset_index()

dfImagesTrain.head()

,index,ImageName,isCancerous,cellType,trainValTest
0,0,./Image_classification_data/patch_images\1.png,0,0,0
1,1,./Image_classification_data/patch_images\10.png,0,0,0
2,3,./Image_classification_data/patch_images\1000.png,1,2,0
3,4,./Image_classification_data/patch_images\10000...,0,1,0
4,5,./Image_classification_data/patch_images\10001...,0,1,0


Note: The definition of the Custom Datasets for both the isCancerous data and the Cell Type data are defined in the pytorch_utility.py file.

Also, rather than loading all the training images and calculating the mean and standard deviation values in here, that was run separately in file 05a.PyTorchGetMeanAndStd.ipynb

Here we can just define the values to use, which shouldn't change unless the data is reloaded and a new train/validation/test split is generated

In [7]:
train_mean, train_std = ptutil.getTrainMeanAndStdTensors()
print(train_mean)
print(train_std)

tensor([0.8035, 0.5909, 0.7640])
tensor([0.1246, 0.1947, 0.1714])


In [8]:
# Create a tranform operation that also normalizes the images according to the mean and standard deviations of the images
transform_normalize = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.ToTensor(), 
    transforms.Normalize(train_mean, train_std)])


In [9]:
cancerous_training_data = None

# Create a custom Dataset for the training and validation data
if useFullData:
    cancerous_training_data = CancerBinaryDataset(isGoogleColab, dfImagesTrain, baseDirectory, transform=transform_normalize)
else:
    # For testing in a small dataset
    dfImagesTrainTest = dfImagesTrain.iloc[range(100), :].reset_index()
    cancerous_training_data = CancerBinaryDataset(isGoogleColab, dfImagesTrainTest, baseDirectory, transform=transform_normalize, target_transform=None)

cancerous_validation_data = CancerBinaryDataset(isGoogleColab, dfImagesVal, baseDirectory, transform=transform_normalize, target_transform=None)

# Create data loaders
cancerous_train_dataloader = DataLoader(cancerous_training_data, batch_size=32, shuffle=True, num_workers=2)
cancerous_val_dataloader = DataLoader(cancerous_validation_data, batch_size=32, shuffle=True, num_workers=2)

Now, create a class for the AlexNet style Convolutional Neural Netowrk. For this basic NN, we will use 3 fully connected layers. The number of features in this will be 27 x 27 x 3, or 2187.

The AlexNet configuration contains 5 convolutional layers, 3 max pooling and an average pooling layer, and in the end it has 3 fully-connected layers, the last of which classifies images in 1000 different classes.

In this, we will use the **ReLU** Activation Function. This is the Rectified Linear Unit function, which allows the function to become non-linear.

Remember that each pooling layer halves both the height and the width of the image, so by using 2 pooling layers, the height and width are 1/4 of the original sizes. 

In [10]:
# Create a class for the Neural Network
class PT_CNN_AlexNet_IsCancerous(nn.Module):

    # In the constructor, initialize the layers to use
    def __init__(self):
        super(PT_CNN_AlexNet_IsCancerous, self).__init__()

        # first, define the subsampling methods. Though they are used multiple times, these are the
        # operations, so only need to be defined once
        # Note: added a second max pooling operation with padding 1 to fix some output sizing issue
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        # define the ReLU Activation method to use
        self.relu = nn.ReLU(inplace=True)

        # define the convolution layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        
        # define the fully connected neural layers
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 2)

    # Create the forward function, which is used in training
    def forward(self, x):

        # print("Init Shape: " + str(x.shape))

        # Process the first 2 convolution layers, applying maxpooling
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool2(x)
        
        # Then process the remaining convolution layers without any pooling
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.relu(self.conv5(x))

        # Then apply a max pool and average pool on the result
        x = self.maxpool2(x)
        x = self.avgpool(x)

        # This should convert to tensors that are acceptable for the input into the NN 3 layers
        x = x.view(x.size(0), 256 * 6 * 6)

        # Now process the 3 layers of the Fully Connected NN
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))        

        # return the result
        return x


Now train the CNN Model.

During training, we will use the following:
- Softmax Cross Entropy Loss as our Loss function. This is a good Loss function that basically converts scores for each class into probabilities
- The Adam Optimizer, which is a version of Gradient Descent
- Initially, just 10 epochs

In [11]:
# set the Learning Rate to use
learning_rate = 0.0001
epochsToUse = 10

net = PT_CNN_AlexNet_IsCancerous()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(epochsToUse):
    print("Starting Epoch " + str(epoch) + "...")
    for i, data in enumerate(cancerous_train_dataloader, 0):
        # Get the inputs
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Perform Forward and Backward propagation then optimize the weights
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

Starting Epoch 0...
Starting Epoch 1...
Starting Epoch 2...
Starting Epoch 3...
Starting Epoch 4...
Starting Epoch 5...
Starting Epoch 6...
Starting Epoch 7...
Starting Epoch 8...
Starting Epoch 9...


Training Time in Nelson's Local Environment on the full data takes a very long time, stopped after 100 minutes. This will need to be done in Colab.

Now Predict according to the validation data and evaluate. While looping through here, we will need to get out the Labels from the data loader, because the order of predictions in the batches do not match the order of the original Target values in the dataset (because we turned Shuffle on)

In [12]:
correct, total = 0,  0
predictions = []

# Set the Neural Network into evaluation (test) mode
net.eval()

step=0

y_val_cancerous = []
y_val_pred_cancerous = []

# Looping through this dataloader essentially processes them in batches of 32 (or whatever the batchsize is configured in the data loader
for i, data in enumerate(cancerous_val_dataloader, 0):
    inputs, labels = data

    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)

    # Loop through the batch, build the lists of the raw label and prediction values
    for j in range(len(labels)):
        y_val_cancerous.append(labels[j].item())
        y_val_pred_cancerous.append(predicted[j].item())

    predictions.append(predicted)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


accuracy = (correct/total) * 100
print("The testing set accuracy of the isCancerous Classification Network is " + str(accuracy) + "%")

The testing set accuracy of the isCancerous Classification Network is 39.57322987390883%


In [13]:
for i in range(3):
    print(predictions[i])

print(len(predictions))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
33


In [14]:
# y_val_pred_cancerous = [item for sublist in y_val_pred_cancerous for item in sublist]
print("Labels")
for i in range(5):
    print(y_val_cancerous[i])

print("Predictions")
for i in range(5):
    print(y_val_pred_cancerous[i])

Labels
1
0
1
1
0
Predictions
0
0
0
0
0


Convert these results into a confusion matrix and then get an F1 Score for comparison

In [15]:
# assuming predictions is just a list of predicted values
cm = confusion_matrix(y_val_cancerous, y_val_pred_cancerous)

print("Binary Classification Results for isCancerous Predictions")
print(classification_report(y_val_cancerous, y_val_pred_cancerous))
print("- Accuracy Score: " + str(accuracy_score(y_val_cancerous, y_val_pred_cancerous)))
print("- Precision Score: " + str(precision_score(y_val_cancerous, y_val_pred_cancerous)))
print("- Recall Score: " + str(recall_score(y_val_cancerous, y_val_pred_cancerous)))
print("- F1 Score: " + str(f1_score(y_val_cancerous, y_val_pred_cancerous)))

Binary Classification Results for isCancerous Predictions
              precision    recall  f1-score   support

           0       0.40      1.00      0.57       408
           1       0.00      0.00      0.00       623

    accuracy                           0.40      1031
   macro avg       0.20      0.50      0.28      1031
weighted avg       0.16      0.40      0.22      1031

- Accuracy Score: 0.3957322987390883
- Precision Score: 0.0
- Recall Score: 0.0
- F1 Score: 0.0


c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

**Results for 10 epochs**

Todo

Now also train a model for CellType Predictions

In [16]:
# Create a custom Dataset for the training and validation data
celltype_training_data = CancerCellTypeDataset(isGoogleColab, dfImagesTrain, baseDirectory, transform=transform_normalize)
celltype_validation_data = CancerCellTypeDataset(isGoogleColab, dfImagesVal, baseDirectory, transform=transform_normalize)

# Create data loaders
celltype_train_dataloader = DataLoader(celltype_training_data, batch_size=32, shuffle=True, num_workers=4)
celltype_val_dataloader = DataLoader(celltype_validation_data, batch_size=32, shuffle=True, num_workers=4)

Create a class for the Cell Type Neural Network model. The structure of the class will be fundamentally the same, only the model will need to output 4 classes

In [17]:
# Create a class for the Neural Network
class PT_CNN_AlexNet_CellType(nn.Module):

    # In the constructor, initialize the layers to use
    def __init__(self):
        super(PT_CNN_AlexNet_CellType, self).__init__()

        # first, define the subsampling methods. Though they are used multiple times, these are the
        # operations, so only need to be defined once
        # Note: added a second max pooling operation with padding 1 to fix some output sizing issue
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        # define the ReLU Activation method to use
        self.relu = nn.ReLU(inplace=True)

        # define the convolution layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        
        # define the fully connected neural layers
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 4)

    # Create the forward function, which is used in training
    def forward(self, x):

        # print("Init Shape: " + str(x.shape))

        # Process the first 2 convolution layers, applying maxpooling
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool2(x)
        
        # Then process the remaining convolution layers without any pooling
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.relu(self.conv5(x))

        # Then apply a max pool and average pool on the result
        x = self.maxpool2(x)
        x = self.avgpool(x)

        # This should convert to tensors that are acceptable for the input into the NN 3 layers
        x = x.view(x.size(0), 256 * 6 * 6)

        # Now process the 3 layers of the Fully Connected NN
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))        

        # return the result
        return x


Now train the Fully Connected Neural Network Model. Use the same configuration (objective function, optimizer etc) as the Binary Classifier

In [18]:
# # set the Learning Rate to use
# learning_rate = 0.0001
# epochsToUse = 10

# net = PT_CNN_AlexNet_CellType()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# for epoch in range(epochsToUse):
#     print("Starting Epoch " + str(epoch) + "...")
#     for i, data in enumerate(celltype_train_dataloader, 0):
#         # Get the inputs
#         inputs, labels = data

#         # Zero the parameter gradients
#         optimizer.zero_grad()

#         # Perform Forward and Backward propagation then optimize the weights
#         outputs = net(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

Starting Epoch 0...
Starting Epoch 1...
Starting Epoch 2...
Starting Epoch 3...
Starting Epoch 4...
Starting Epoch 5...
Starting Epoch 6...
Starting Epoch 7...


Predict on the Validation data and evaluate the results

In [ ]:
# correct, total = 0,  0
# predictions = []

# # Set the Neural Network into evaluation (test) mode
# net.eval()

# step=0

# y_val_celltype = []
# y_val_pred_celltype = []

# # Looping through this dataloader essentially processes them in batches of 32 (or whatever the batchsize is configured in the data loader
# for i, data in enumerate(celltype_val_dataloader, 0):
#     inputs, labels = data

#     # This should convert the image tensors into vectors
#     inputs = inputs.view(-1, 27 * 27 * 3)

#     outputs = net(inputs)
#     _, predicted = torch.max(outputs.data, 1)
    
#     # print(labels)
#     # print(predicted)  
#     # print(len(labels))
#     # print(len(predicted))

#     # Loop through the batch, build the lists of the raw label and prediction values
#     for j in range(len(labels)):
#         y_val_celltype.append(labels[j].item())
#         y_val_pred_celltype.append(predicted[j].item())

#     predictions.append(predicted)
#     total += labels.size(0)
#     correct += (predicted == labels).sum().item()


# accuracy = (correct/total) * 100
# print("The testing set accuracy of the CellType Classification Network is " + str(accuracy) + "%")

Convert these results into a confusion matrix and then get an F1 Score for comparison

In [ ]:
# # assuming predictions is just a list of predicted values
# cm = confusion_matrix(y_val_celltype, y_val_pred_celltype)

# print("CellType Multi-class Classification Results for Cell Type Predictions")
# print(classification_report(y_val_celltype, y_val_pred_celltype))
# print("- Accuracy Score: " + str(accuracy_score(y_val_celltype, y_val_pred_celltype)))
# print("- Precision Score: " + str(precision_score(y_val_celltype, y_val_pred_celltype, average="micro")))
# print("- Recall Score: " + str(recall_score(y_val_celltype, y_val_pred_celltype, average="micro")))
# print("- F1 Score: " + str(f1_score(y_val_celltype, y_val_pred_celltype, average="micro")))

**Results for 10 epochs:**

Todo